In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params, gaussian_kernel
from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.models import unet_builder as bare_model_builder
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels

In [2]:
PROJECT_NAME = 'unet'

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 500
BATCH_SIZE = 8096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all_original, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all_original[(2, 11)].shape

((22412, 24), (22412,))

In [4]:
X_train, X_val, y_train_original, y_val_original = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train_original = np.concatenate(list(y_train_original.values()))
X_val = np.concatenate(list(X_val.values()))
y_val_original = np.concatenate(list(y_val_original.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train_original = X_train[train_idx], y_train_original[train_idx]
X_val, y_val_original = X_val[val_idx], y_val_original[val_idx]

X_train.shape, y_train_original.shape, X_val.shape, y_val_original.shape

((85651, 24), (85651,), (21416, 24), (21416,))

In [5]:
y_all = {key: np.array([gaussian_kernel(y) for y in val]) for key, val in y_all_original.items()}
y_train = np.array([gaussian_kernel(y) for y in y_train_original])
y_val = np.array([gaussian_kernel(y) for y in y_val_original])

y_all_original[(2, 11)].shape, y_train.shape, y_val.shape

((22412,), (85651, 24), (21416, 24))

# Model

In [6]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_unet_depth = hp.Int("unet_depth", min_value=0, max_value=3, step=1, default=2)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8, 16], default=4)
    hp_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_unet_depth, hp_n_conv_layers, hp_filters_mult, hp_spatial_dropout,
                               hp_batch_normalization, hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [7]:
model_builder(kt.HyperParameters()).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       96          ['reshape[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 12, 32)       0           ['conv1d[0][0]']                 
                                                                                              

In [8]:
model_callbacks = [
    callbacks.EarlyStopping(patience=30, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=5)
]

# Bayesian tuner

In [9]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 01m 08s]
val_loss: 15.744112014770508

Best val_loss So Far: 12.82594633102417
Total elapsed time: 01h 46m 17s


# Results

In [10]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 11521
========== Model 1 ==========
{'unet_depth': 2, 'n_conv_layers': 2, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 7793
========== Model 2 ==========
{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 2962785
========== Model 3 ==========
{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 65025
========== Model 4 ==========
{'une

# Cross-validation for top 5 models

In [11]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 11521
Got score: 12.6264 (12.6362, 12.6166)
Got score: 12.7063 (12.7129, 12.6997)
Got score: 12.7228 (12.6793, 12.7664)
Got score: 12.7094 (12.6711, 12.7477)
Got score: 12.5517 (12.5656, 12.5377)


{'unet_depth': 2, 'n_conv_layers': 2, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 7793
Got score: 12.6006 (12.5799, 12.6213)
Got score: 12.7058 (12.7082, 12.7035)
Got score: 12.6569 (12.6534, 12.6605)
Got score: 12.6841 (12.7440, 12.6241)
Got score: 12.5578 (12.5769, 12.5387)


{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 2962785
Got score: 12.5271 (12.4548, 12.5994)
Got score: 12.6249 (12.4910, 12.7587)
Got score: 12.6702 (12.7953, 12.5450)
Got score: 12.7159 (12.7439, 12.6878)
Got score: 12.5224 (12.4779, 12.5670)


{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 65025
Got score: 12.6097 (12.6320, 12.5873)
Got score: 12.7702 (12.7580, 12.7824)
Got score: 12.6723 (12.6826, 12.6621)
Got score: 12.7337 (12.7739, 12.6935)
Got score: 12.5776 (12.5395, 12.6157)


{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 47137
Got score: 13.0541 (12.7773, 13.3310)
Got score: 13.4585 (13.9714, 12.9455)
Got score: 12.8450 (12.6129, 13.0772)
Got score: 12.8196 (12.9409, 12.6983)
Got score: 12.8649 (12.9811, 12.7486)


In [12]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,12.66,0.07,11521
1,12.64,0.05,7793
2,12.61,0.08,2962785
3,12.67,0.07,65025
4,13.01,0.24,47137
